In [1]:
import warnings
warnings.filterwarnings('ignore')
#Import necesary modules
#Use shorter names (np, pd, plt) instead of full (numpy, pandas, matplotlib.pylot) for convenience
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import numpy.ma as ma
import h5py
import s3fs
import matplotlib.pyplot as plt
import netCDF4
import xarray as xr
import load_in_beam

# Use seasborn for nicer looking inline plots if available 
#import seaborn as sns
#sns.set(context='notebook', style='darkgrid')
#st = axes_style("whitegrid")


In [2]:
ATL07file='processed_ATL07-01_20181130235806_09680101_001_01.h5'
# ATL07file='processed_ATL07-01_20181206115828_10520101_001_01.h5'
localFilePath='/home/jovyan/Data/'+ATL07file

print(localFilePath)

ds1r = load_in_beam.read_beam(filename=localFilePath,beam='gt1r')

/home/jovyan/Data/processed_ATL07-01_20181130235806_09680101_001_01.h5
/home/jovyan/Data/processed_ATL07-01_20181130235806_09680101_001_01.h5


Take a look at the dataset we made 

<xarray.Dataset>
Dimensions:       (segs: 252561)
Coordinates:
    lon           (segs) float64 -171.2 -171.2 -171.2 ... 30.86 30.85 30.79
    lat           (segs) float64 72.58 72.58 72.58 72.58 ... 83.2 83.2 83.18
    time          (segs) datetime64[ns] 2018-11-30T23:55:29 ... 2018-12-01T00:01:45
    delta_time    (segs) float64 2.886e+07 2.886e+07 ... 2.886e+07 2.886e+07
    lon360        (segs) float64 188.8 188.8 188.8 188.8 ... 30.86 30.85 30.79
  * segs          (segs) int64 0 1 2 3 4 ... 252556 252557 252558 252559 252560
Data variables:
    seg_dist      (segs) float64 8.099e+06 8.099e+06 ... 1.076e+07 1.077e+07
    height        (segs) float32 3.4028235e+38 3.4028235e+38 ... 3.4028235e+38
    mss           (segs) float32 -0.44165832 -0.44165832 ... 20.844654 20.875843
    seg_length    (segs) float